<a href="https://colab.research.google.com/github/allyoushawn/jupyter_notebook_projects/blob/main/ml_misc/LLM_rl_training_example.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

#RL training GPT-2

In [ ]:
!pip install transformers datasets accelerate

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 363.4/363.4 MB 4.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 13.8/13.8 MB 35.6 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 24.6/24.6 MB 22.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 883.7/883.7 kB 22.1 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 664.8/664.8 MB 2.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 211.5/211.5 MB 3.0 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 56.3/56.3 MB 10.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 127.9/127.9 MB 9.3 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 207.5/207.5 MB 4.7 MB/s eta 0:00:00
   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 21.1/21.1 MB 52.3 MB/s eta 0:00:00
  Attempting uninstall: nvidia-nvjitlink-cu12
    Found existing installation: nvidia-nvjitlink-cu12 12.5.82
    Uninstalling nvidia-nvjitlink-cu12-12.5.82:
      Successfully uninstalled nvidia-nvjitlin

In [ ]:
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import random


In [ ]:
# prompt: Check if cuda is available to use

device = 'cuda' if torch.cuda.is_available() else 'cpu'
print(f"Using device: {device}")


Using device: cpu


In [ ]:
tokenizer = GPT2Tokenizer.from_pretrained("gpt2")
model = GPT2LMHeadModel.from_pretrained("gpt2")
model.to(device)
model.train()

In [ ]:
# Add padding token if missing
tokenizer.pad_token = tokenizer.eos_token
model.config.pad_token_id = tokenizer.pad_token_id

# Reward function: +1 if 'cat' is in the generated text, else 0
def reward_fn(text):
    return 1.0 if "cat" in text.lower() else 0.0


In [ ]:
# prompt: Show the number of parameters of the model

print(f'The model has {sum(p.numel() for p in model.parameters())} parameters')

The model has 124439808 parameters


In [ ]:
def sample_text(prompt="The animal is", max_len=20):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=inputs['input_ids'].shape[1] + max_len,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)
loss_fn = nn.CrossEntropyLoss()


In [ ]:
def reinforce_train_step(prompt="The animal is"):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_len = inputs['input_ids'].shape[1]

    outputs = model.generate(
        **inputs,
        max_length=input_len + 20,
        do_sample=True,
        return_dict_in_generate=True,
        output_scores=True,
        output_hidden_states=False
    )

    sequences = outputs.sequences  # shape: (1, seq_len)
    scores = outputs.scores        # list of (1, vocab_size) tensors

    generated_tokens = sequences[0, input_len:]  # only generated part
    if len(scores) != len(generated_tokens):
        print("⚠️ Mismatch between scores and generated tokens")
        return "error", 0.0

    log_probs = torch.stack(scores, dim=0).log_softmax(dim=-1).squeeze(1)  # shape: (gen_len, vocab)
    chosen_token_ids = generated_tokens
    vocab_size = log_probs.shape[-1]

    if (chosen_token_ids >= vocab_size).any():
        print("⚠️ Token ID out of bounds; clipping")
        chosen_token_ids = chosen_token_ids.clamp(max=vocab_size - 1)

    chosen_log_probs = log_probs[range(len(chosen_token_ids)), chosen_token_ids]

    # Reward
    generated_text = tokenizer.decode(sequences[0], skip_special_tokens=True)
    R = reward_fn(generated_text)

    # REINFORCE loss
    loss = -R * chosen_log_probs.mean()
    loss.backward()
    optimizer.step()
    optimizer.zero_grad()

    return generated_text, R


In [ ]:
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import random

# Ensure necessary imports are present if running this function standalone
# from IPython import get_ipython
# from IPython.display import display
# !pip install transformers datasets accelerate
# import torch
# from torch import nn
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# import random

# Assume 'device', 'tokenizer', 'model', 'reward_fn', and 'optimizer' are defined

def reinforce_train_step(prompt="The animal is"):
    # Initial prompt input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_len = inputs['input_ids'].shape[1]

    # Generate sequence using the model
    outputs = model.generate(
        **inputs,
        max_length=input_len + 20,
        do_sample=True,
        return_dict_in_generate=True,
        output_scores=False,  # No longer need scores from generate
        output_hidden_states=False
    )

    generated_sequences = outputs.sequences  # shape: (1, seq_len)

    # Calculate log probabilities for the *generated* sequence
    # We need to run a forward pass on the model with the generated sequence
    # to get the logits with gradient tracking.
    full_outputs = model(generated_sequences, labels=generated_sequences) # Use labels for automatic loss calculation if needed, or calculate manually

    # Extract logits for the generated tokens part
    # The logits are for predicting the *next* token given the current sequence
    # So, logits[:, :-1, :] correspond to predicting tokens at indices 1 to end
    # and we are interested in the probabilities of the tokens that were actually generated.
    logits = full_outputs.logits[:, :-1, :]  # shape: (1, seq_len-1, vocab_size)
    target_tokens = generated_sequences[:, 1:] # shape: (1, seq_len-1) - these are the tokens predicted by the logits

    # Calculate log probabilities of the chosen tokens
    log_probs_all = torch.log_softmax(logits, dim=-1) # shape: (1, seq_len-1, vocab_size)

    # Select the log probabilities corresponding to the generated tokens
    # Flatten the tensors for easier indexing
    log_probs_flat = log_probs_all.view(-1, log_probs_all.size(-1)) # shape: (seq_len-1, vocab_size)
    target_tokens_flat = target_tokens.view(-1) # shape: (seq_len-1)

    # Filter to get only the log probabilities for the generated part (after the prompt)
    # Need to adjust indices based on the prompt length
    log_probs_generated_flat = log_probs_flat[input_len-1:] # shape: (gen_len, vocab_size)
    target_tokens_generated_flat = target_tokens_flat[input_len-1:] # shape: (gen_len)

    # Select the log probabilities for the actually chosen tokens in the generated part
    chosen_log_probs = log_probs_generated_flat[range(len(target_tokens_generated_flat)), target_tokens_generated_flat]

    # Reward
    generated_text = tokenizer.decode(generated_sequences[0], skip_special_tokens=True)
    R = reward_fn(generated_text)

    # REINFORCE loss
    # We want to maximize the expected reward, which is equivalent to minimizing -expected_reward.
    # The expected reward is R * sum(log_probs of chosen tokens).
    # However, since we are doing batch gradient descent and have one sample,
    # the gradient is proportional to R * gradient(sum(log_probs)).
    # A common simplification in REINFORCE is to use -R * sum(log_probs) as the loss.
    # Here, we are using the mean of log probs over the generated sequence length.
    loss = -R * chosen_log_probs.mean()

    # Backpropagation and optimizer step
    optimizer.zero_grad() # Zero gradients before backward pass
    loss.backward()
    optimizer.step()


    return generated_text, R

In [ ]:
print("Before training")
for _ in range(5):
    print("→", sample_text())

print("\nTraining...")
for step in range(100):
    text, r = reinforce_train_step()
    if step % 10 == 0:
        print(f"[Step {step}] Reward: {r:.2f} | Text: {text}")

print("\nAfter training")
for _ in range(5):
    print("→", sample_text())


Before training
→ The animal is usually the small horn or slightly larger than it appears on the skin. The small animal can also be
→ The animal is not a real creature but is in fact a sentient animal, with its very shape and structure. There
→ The animal is under constant threat of being injured, shot or otherwise harmed, for which all the owners' legal counsel
→ The animal is a tiny house animal, and its only living creature is a human one, which is why this is


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


→ The animal is actually just a human. I mean when you look at this, it's almost exactly the same,

Training...


`loss_type=None` was set in the config but it is unrecognised.Using the default loss: `ForCausalLMLoss`.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Step 0] Reward: 0.00 | Text: The animal is at the level, he says, only this time when the other has been in an unexpected position.


...

[Step 80] Reward: 1.00 | Text: The animal is the cat".


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.


[Step 90] Reward: 1.00 | Text: The animal is the cat".


Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:50256 for open-end generation.



After training
→ The animal is the cat".
→ The animal is the cat"
→ The animal is the cat".
→ The animal is the cat".
→ The animal is the cat".


# RL with LLM using LoRA from huggingface

In [ ]:
pip install transformers peft accelerate bitsandbytes

   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 67.0/67.0 MB 12.1 MB/s eta 0:00:00


In [ ]:
import torch
from transformers import AutoTokenizer, AutoModelForCausalLM
from peft import get_peft_model, LoraConfig, TaskType
import torch.nn.functional as F

# Use a small model like Falcon-RW-1B or GPT2 for fast CPU training
model_name = "tiiuae/falcon-rw-1b"

tokenizer = AutoTokenizer.from_pretrained(model_name, padding_side="right")
if tokenizer.pad_token is None:
    tokenizer.pad_token = tokenizer.eos_token

base_model = AutoModelForCausalLM.from_pretrained(model_name, device_map=None)

# Enable adapter with LoRA
lora_config = LoraConfig(
    r=8,
    lora_alpha=16,
    target_modules=["query_key_value", "dense", "dense_h_to_4h"],  # Adjust based on model
    lora_dropout=0.1,
    bias="none",
    task_type=TaskType.CAUSAL_LM
)

model = get_peft_model(base_model, lora_config)
model.config.pad_token_id = tokenizer.pad_token_id
device = torch.device("cpu")
model = model.to(device)
model.train()


In [ ]:
def reward_fn(text):
    return 1.0 if "cat" in text.lower() else 0.0


In [ ]:
def sample_text(prompt="The animal is", max_len=20):
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    outputs = model.generate(
        **inputs,
        max_length=inputs['input_ids'].shape[1] + max_len,
        do_sample=True,
        top_k=50,
        top_p=0.95,
        temperature=1.0,
        pad_token_id=tokenizer.pad_token_id
    )
    return tokenizer.decode(outputs[0], skip_special_tokens=True)


In [ ]:
optimizer = torch.optim.Adam(model.parameters(), lr=5e-5)



In [ ]:
import torch
from torch import nn
from transformers import GPT2Tokenizer, GPT2LMHeadModel
import random

# Ensure necessary imports are present if running this function standalone
# from IPython import get_ipython
# from IPython.display import display
# !pip install transformers datasets accelerate
# import torch
# from torch import nn
# from transformers import GPT2Tokenizer, GPT2LMHeadModel
# import random

# Assume 'device', 'tokenizer', 'model', 'reward_fn', and 'optimizer' are defined

def reinforce_train_step(prompt="The animal is"):
    # Initial prompt input
    inputs = tokenizer(prompt, return_tensors="pt").to(device)
    input_len = inputs['input_ids'].shape[1]

    # Generate sequence using the model
    outputs = model.generate(
        **inputs,
        max_length=input_len + 20,
        do_sample=True,
        return_dict_in_generate=True,
        output_scores=False,  # No longer need scores from generate
        output_hidden_states=False
    )

    generated_sequences = outputs.sequences  # shape: (1, seq_len)

    # Calculate log probabilities for the *generated* sequence
    # We need to run a forward pass on the model with the generated sequence
    # to get the logits with gradient tracking.
    full_outputs = model(generated_sequences, labels=generated_sequences) # Use labels for automatic loss calculation if needed, or calculate manually

    # Extract logits for the generated tokens part
    # The logits are for predicting the *next* token given the current sequence
    # So, logits[:, :-1, :] correspond to predicting tokens at indices 1 to end
    # and we are interested in the probabilities of the tokens that were actually generated.
    logits = full_outputs.logits[:, :-1, :]  # shape: (1, seq_len-1, vocab_size)
    target_tokens = generated_sequences[:, 1:] # shape: (1, seq_len-1) - these are the tokens predicted by the logits

    # Calculate log probabilities of the chosen tokens
    log_probs_all = torch.log_softmax(logits, dim=-1) # shape: (1, seq_len-1, vocab_size)

    # Select the log probabilities corresponding to the generated tokens
    # Flatten the tensors for easier indexing
    log_probs_flat = log_probs_all.view(-1, log_probs_all.size(-1)) # shape: (seq_len-1, vocab_size)
    target_tokens_flat = target_tokens.view(-1) # shape: (seq_len-1)

    # Filter to get only the log probabilities for the generated part (after the prompt)
    # Need to adjust indices based on the prompt length
    log_probs_generated_flat = log_probs_flat[input_len-1:] # shape: (gen_len, vocab_size)
    target_tokens_generated_flat = target_tokens_flat[input_len-1:] # shape: (gen_len)

    # Select the log probabilities for the actually chosen tokens in the generated part
    chosen_log_probs = log_probs_generated_flat[range(len(target_tokens_generated_flat)), target_tokens_generated_flat]

    # Reward
    generated_text = tokenizer.decode(generated_sequences[0], skip_special_tokens=True)
    R = reward_fn(generated_text)

    # REINFORCE loss
    # We want to maximize the expected reward, which is equivalent to minimizing -expected_reward.
    # The expected reward is R * sum(log_probs of chosen tokens).
    # However, since we are doing batch gradient descent and have one sample,
    # the gradient is proportional to R * gradient(sum(log_probs)).
    # A common simplification in REINFORCE is to use -R * sum(log_probs) as the loss.
    # Here, we are using the mean of log probs over the generated sequence length.
    loss = -R * chosen_log_probs.mean()

    # Backpropagation and optimizer step
    optimizer.zero_grad() # Zero gradients before backward pass
    loss.backward()
    optimizer.step()


    return generated_text, R

In [ ]:
print("Before training")
for _ in range(3):
    print("→", sample_text())

print("\nTraining...")
for step in range(30):  # Increase to 100+ to see better behavior
    text, r = reinforce_train_step()
    if step % 5 == 0:
        print(f"[Step {step}] Reward: {r:.2f} | Text: {text}")

print("\nAfter training")
for _ in range(3):
    print("→", sample_text())


Before training
→ The animal is an adult. The animal is an adult female. The dog is a young dog. If the dog
→ The animal is also likely to become aggressive if it believes it is threatened, and a vicious animal will usually defend the


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


→ The animal is not for sale. This is a free service provided by the
University of Kentucky
.
K

Training...


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[Step 0] Reward: 0.00 | Text: The animal is not the only one who's been affected by the shutdown. So have veterinarians.
On a


...

[Step 190] Reward: 1.00 | Text: The animal is cats birdbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirds


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.


[Step 195] Reward: 1.00 | Text: The animal is cats birdbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirds


Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.
Setting `pad_token_id` to `eos_token_id`:2 for open-end generation.



After training
→ The animal is cats birdbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirds
→ The animal is cats birdbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirds
→ The animal is catsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirdsbirds
